In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.metrics import accuracy_score, confusion_matrix

import seaborn as sns

### Import data

In [ ]:
file = "pidTraining_LE.hdf5"
filename = "/home/ech18004/PID/data/" + file

train = pd.read_hdf(filename, 'event1')

file = "pidTest_LE.hdf5"
filename = "/home/ech18004/PID/data/" + file

test = pd.read_hdf(filename, 'event1')

### Print the labels of the data (the training and test data have the same labels)

In [ ]:
for i in range(len(train.columns)):
    print(i, train.columns[i]) 

### See what particle types are avialable

In [ ]:
ptypes = np.array([np.int64(train['ptype'][i]) for i in range(0, len(train), 80000)])
ptype = {22:0, 130:1, 2112:2, 2212:3, -2212:4, 321:5, -321:6, 11:7, -11:8, 211:9, -211:10}
val_to_p = {22:r'$\gamma$', 130:r'$K_{L}^{0}$', 2112:r'$n$', 2212:r'$p$', -2212:r'$\bar{p}$', 321:r'$K^{+}$', -321:r'$K^{-}$', 11:r'$e^{-}$', -11:r'$e^{+}$', 211:r'$\pi^{+}$', -211:r'$\pi^{-}$'}
ptypes

### Remove unwanted labels from test data

In [ ]:
testnp = pd.DataFrame.to_numpy(test)
testind = np.array([i for i in range(len(testnp)) if np.float(testnp[i][0]) == testnp[i][-1]])
testout = test.loc[testind]
testout = test.drop(['ptype', 'group'], axis=1)

### Separate data into numpy arrays and set nans to a float value

In [ ]:
repval = 0
listvals = np.array([27, 28, 31, 32])
listvals = ['posShower', 'posTrack', 'posTOF', 'tFlights']

train = train.replace(np.nan, repval)
testout = testout.replace(np.nan, repval)

train = train.drop(listvals, axis=1)
testout = testout.drop(listvals, axis=1)

trainx, trainy = pd.DataFrame.to_numpy(train.drop('ptype', axis=1)), pd.DataFrame.to_numpy(train['ptype'])
trainy = trainy.astype(np.int64)
testx, testy = pd.DataFrame.to_numpy(testout.drop('true ptype', axis=1)), pd.DataFrame.to_numpy(test['true ptype'])

trainy = np.array([ptype[trainy[i]] for i in range(len(trainy))])
testy = np.array([ptype[testy[i]] for i in range(len(testy))])

### Convert to tensorflow datasets

In [ ]:
tf_train = tf.data.Dataset.from_tensor_slices((trainx, trainy)).cache()
tf_test = tf.data.Dataset.from_tensor_slices((testx, testy)).cache()

tf_train = tf_train.shuffle(len(tf_train))

tf_train = tf_train.batch(128)
tf_test = tf_test.batch(128)

tf_train = tf_train.prefetch(tf.data.AUTOTUNE)
tf_test = tf_test.prefetch(tf.data.AUTOTUNE)

# Create and Train a DNN

### Create model

In [ ]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten(input_shape=(253,))) ###Only need to flatten input if it was not already previously flattened
model.add(tf.keras.layers.Dense(253, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(len(ptypes), activation = 'sigmoid'))

#model.compile(optimizer='adam',
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])

model.compile(optimizer = tf.keras.optimizers.Adam(), 
              loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics = [tf.keras.metrics.SparseCategoricalAccuracy()],)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01, patience=5) #, start_from_epoch=10)

### Train model

In [ ]:
model.fit(tf_train, 
          epochs=50, 
          validation_data=tf_test,
          callbacks=[callback],
          verbose = 1)

### Use model to predict on the test data and find the accuracy of the model

In [ ]:
pred = model.predict(testx)
predarr = np.array([np.argmax(pred[i]) for i in range(len(pred))])
acc = accuracy_score(testy, predarr)
acc

### Creat a confusion matrix for the model created above

In [ ]:
cm = confusion_matrix(testy, predarr, normalize='true')
#cm = np.array([cm[i]/len(testy[testy == i]) for i in range(len(cm))])
cm = np.round(cm, decimals=3)

fig = plt.figure()
fig.set_size_inches(10, 10)
ax = fig.add_subplot(111)


sns.heatmap(cm, annot=True, fmt='g', ax=ax);


ax.set_xlabel('Predicted Particle', fontsize=15);ax.set_ylabel('Generated Particle', fontsize=15); 
ax.set_title(f'PID Confusion Matrix with a Total Accuracy of {round(acc, 4)}', fontsize=17); 
ax.xaxis.set_ticklabels([r'$\gamma$', r'$K_{L}^{0}$', r'$n$', r'$p$', r'$\bar{p}$', 
                         r'$K^{+}$', r'$K^{-}$', r'$e^{-}$', r'$e^{+}$', r'$\pi^{+}$', r'$\pi^{-}$'], fontsize=15);
ax.yaxis.set_ticklabels([r'$\gamma$', r'$K_{L}^{0}$', r'$n$', r'$p$', r'$\bar{p}$', 
                         r'$K^{+}$', r'$K^{-}$', r'$e^{-}$', r'$e^{+}$', r'$\pi^{+}$', r'$\pi^{-}$'], fontsize=15);

# See how well the model worked on data that had no hypotheses that matched the true ptype

### Code to find the indices of each test data with correct particle hypothesis

In [ ]:
testnp = pd.DataFrame.to_numpy(test)
iter = 0
no_id = np.array([])
for i in range(int(len(train)/2)):        #Divided by 2 since the training dataset is 2x longer than the test dataset
    group_list = np.where(test['group'] == i)[0]
    count = 0
    for j in range(len(group_list)):
        if np.float(testnp[iter][0]) == testnp[iter][-1]:
            count += 1
        iter += 1
    if count == 0:
        no_id = np.append(no_id, group_list)

no_id = no_id.astype(np.int64)

### Calcualte accuracy for the vecotrs with incorrect hypotheses

In [ ]:
testy_noID = np.array([testy[i] for i in no_id])
predarr_noID = np.array([predarr[i] for i in no_id])

acc = accuracy_score(testy_noID, predarr_noID)
print(acc)

### Add 1 label for each ptype (so confusion_matrix works)

In [ ]:
for i in range(3):
    testy_noID = np.append(testy_noID, testy[testy == i][0])
    predarr_noID = np.append(predarr_noID, predarr[predarr == i][0])

### Print a confusion matrix for the vectors with incorrect hypotheses

In [ ]:
cm = confusion_matrix(testy_noID, predarr_noID, normalize='true')
#cm = np.array([cm[i]/len(testy_noID[testy_noID == i]) for i in range(len(cm))])
cm = np.round(cm, decimals=3)

for i in range(3):         ##Set the extra values to zero
    cm[i][i] = 0

fig = plt.figure()
fig.set_size_inches(10, 10)
ax = fig.add_subplot(111)


sns.heatmap(cm, annot=True, fmt='g', ax=ax);


ax.set_xlabel('Predicted Particle', fontsize=15);ax.set_ylabel('Generated Particle', fontsize=15); 
ax.set_title(f'PID Confusion Matrix with a Total Accuracy of {round(acc, 4)}', fontsize=17); 
ax.xaxis.set_ticklabels([r'$\gamma$', r'$K_{L}^{0}$', r'$n$', r'$p$', r'$\bar{p}$', 
                         r'$K^{+}$', r'$K^{-}$', r'$e^{-}$', r'$e^{+}$', r'$\pi^{+}$', r'$\pi^{-}$'], fontsize=15);
ax.yaxis.set_ticklabels([r'$\gamma$', r'$K_{L}^{0}$', r'$n$', r'$p$', r'$\bar{p}$', 
                         r'$K^{+}$', r'$K^{-}$', r'$e^{-}$', r'$e^{+}$', r'$\pi^{+}$', r'$\pi^{-}$'], fontsize=15);